<a href="https://colab.research.google.com/github/Kwannn666/GAN_AI/blob/main/Q36134255_HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 0. Reflection 的任務設計

##### 🌟 任務說明：職場 Email 優化小助理

**🎯 流程說明：**
1. 使用者輸入email的出版草稿內容
2. `model_writer` 將輸入內容改寫成正式、禮貌、格式正確的Email版本
3. `model_reviewer` 檢查第一版Email的禮貌性、語氣自然度、段落格式，並提供改進建議
4. `model_writer` 根據建議，再次改寫出更正式、得體、流暢的第二版Email
5. Gradio呈現：三個欄位輸出（第一版正式Email、修改建議、第二版優化Email）

#### 1. 安裝必要套件

In [ ]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.0 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.11.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00


引入標準套件

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

引入必要套件

In [ ]:
import os
from google.colab import userdata
import aisuite as ai
import gradio as gr

#### 2. 讀入金鑰

這邊分別使用了OpenAI以及Groq的金鑰

OpenAI 的模型使用 gpt-4o

Groq 的模型使用 llama3-70b-8192

In [ ]:
#【使用 OpenAI】
api_key = userdata.get('OpenAI')
os.environ['OPENAI_API_KEY']=api_key
provider = "openai"
model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

#### 3. 基本的設定

#####Provider 設定

Writer 設定 ( Groq 的 llama3-70b-8192 模型 )

In [ ]:
provider_writer = "groq"
model_writer="llama3-70b-8192"

Reviewer 設定 ( OpenAI 的 gpt-4o 模型 )

In [ ]:
provider_reviewer = "openai"
model_reviewer = "gpt-4o"

#### 4. 定義 Chat 函數

In [ ]:
def reply(system="請用台灣習慣(繁體中文)的中文回覆。", prompt="hi", provider="groq", model="llama3-70b-8192"):
    client = ai.Client()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)
    return response.choices[0].message.content

####  5.  Reflection 人設設定

作者人設設定

你是一位專業的商業Email寫手，擅長將簡單的內容整理成正式、禮貌且語氣得體的Email格式。請保持尊重、清晰、有禮貌且邏輯的職場用語，符合台灣職場文化(科技業導向)，並以自然中文(繁體中文)回應。

In [ ]:
system_writer = "你是一位專業的商業Email寫手，擅長將簡單的內容整理成正式、禮貌且語氣得體的Email格式。請保持尊重、清晰、有禮貌且邏輯的職場用語，符合台灣職場文化(科技業導向)，並以自然中文(繁體中文)回應。"

審查員人設設定

你是一位職場溝通專家，擅長審閱Email文案。請檢查禮貌性、語氣自然度、段落格式以及文章邏輯清晰度，並提供具體改進建議。請以台灣職場習慣的中文(繁體中文)回覆。

In [ ]:
system_reviewer = "你是一位職場溝通專家，擅長審閱Email文案。請檢查禮貌性、語氣自然度、段落格式以及文章邏輯清晰度，並提供具體改進建議。請以台灣職場習慣的中文(繁體中文)回覆。"

Reflection 任務設計

In [ ]:
def reflect_email(prompt):
    # Step 1: Writer 初稿
    first_version = reply(
        system_writer,
        prompt,
        provider=provider_writer,
        model=model_writer
    )

    # Reviewer 提供建議
    suggestion = reply(
        system_reviewer,
        first_version,
        provider=provider_reviewer,
        model=model_reviewer
    )

    # Step 3: Writer 再寫一次（根據建議）
    second_prompt = f"請根據以下建議改寫這封Email，使其更自然、正式且得體：\n建議：{suggestion}\n原始內容：{first_version}"
    second_version = reply(
        system_writer,
        second_prompt,
        provider=provider_writer,
        model=model_writer
    )

    return first_version, suggestion, second_version


### 4. 用 Gradio 打造你的對話機器人 Web App!

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("#  職場 Email 優化小助理（Reflection Agent）")
    gr.Markdown("請輸入你草擬的Email內容，我們將協助你優化成更正式、有禮貌且邏輯清晰的版本。")

    user_input = gr.Textbox(label="請輸入草擬的Email內容")
    btn = gr.Button(" 生成正式Email & 修正建議")

    with gr.Row():
        out1 = gr.Textbox(label=" 第一版正式Email (model_writer)")
        out2 = gr.Textbox(label=" 修改建議 (model_reviewer)")
        out3 = gr.Textbox(label=" 第二版優化整合後Email (model_writer 改寫)")

    btn.click(reflect_email, inputs=[user_input], outputs=[out1, out2, out3])


In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e73b123eb2a8bd8129.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
